# Automated Reasoning Policy Refinement Playground

Refining a Automated Reasoning policy is an important step in improving the soundness of Automated Reasoning Checks. Policy refinement involves modifications to rules, variables and types of the Automated Reasoning policy. This can be done through direct updates to the rules, variables or types. 
It can also be done through natual language feedback or by generating scenarios to test the rules, and then providing feedback from the test results of the generated scenarios.

This notebook demonstrates how to generate scenarios from the policy, run tests using the generated scenario, and update rules, types and variables of the automated reasoning policy using the AWS Bedrock APIs. It includes the following functionality:

1. Creating annotations
    1. Using Scenario Generation to identify which annotations are required through automated testing
2. Generating a new Policy Definition using the annotations
3. Update the Automated Reasoning Policy with the updated Policy definitions


### Pre-requisites for running this notebook
This notebook uses the medical policy created in Lab1 - If you have not set up Automated Reasoning policy for the sample medical policy executed that, first run Lab1.

# Setup

In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import boto3
import uuid
import pandas as pd
from IPython.display import display, HTML, JSON
import pprint

In [ ]:
# Create the Bedrock client
my_session = boto3.session.Session()
REGION_NAME = my_session.region_name
print(f'The region is {REGION_NAME}')

runtime_client = my_session.client('bedrock-runtime', region_name=REGION_NAME)
bedrock_client = my_session.client('bedrock', region_name=REGION_NAME)

# Automated Reasoning Policy Refinement

Policy refinement is an important step in validating that the rules captured in the Automated Reasoning (AR) Policy is accurate, complete and represents the source document/ truth well. This step is crucial in improving the soundness of Automated Reasoning checks.

Steps:
1. Create an annotation
2. Start the automated reasoning workflow with the annotation as a policy repair asset
3. Check the result of the workflow and analyse the assets: build log and quality results
4. Now we can update the automated reasoning policy 

#### Before executing this lab, retrieve the ARN of the Policy created and provide it below

In [ ]:
# Provide the policy arn for which you will create a test case
# TODO
policy_arn="" # Update with the policy created in Lab 1

# Retrieve all build workflows associated with the specified policy
list_build_workflows_response = bedrock_client.list_automated_reasoning_policy_build_workflows(policyArn=policy_arn)

# Extract the build workflow ID from the first workflow in the response
# This ID is required to run test cases against the policy
build_workflow_id = list_build_workflows_response['automatedReasoningPolicyBuildWorkflowSummaries'][0]['buildWorkflowId']

### What are annotations?

An annotation is a addition/update/modification performed on the Automated Reasoning Policy through addition/ modification/ deletion of rules, variables and custom types. 

The allowed annotations are:
1. `` addType ``: Add a new type by passing in a name, description and value of the new type
2. `` updateType ``: Update a type by passing in the name of the type to be updated, a new name, description and value
3. `` deleteType ``: Delete a type by passing in the name of the type to be deleted
4. `` addVariable ``: Add a new variable by passing in a name, description and type of the new variable
5. `` updateVariable ``: Update a variable by passing in the name of the variable to be updated, a new name and description
6. `` addType ``: Add a new type
7. `` addType ``: Add a new type
8. `` addType ``: Add a new type
9. `` addType ``: Add a new type
10. `` addType ``: Add a new type

In [ ]:
def create_annotation(annotation_type, **kwargs):
    """
    Create an Automated Reasoning Policy Annotation dynamically
    
    Args:
        annotation_type (str): Type of annotation to create
        **kwargs: Keyword arguments specific to the annotation type
    
    Returns:
        dict: Annotation dictionary
    
    Supported Annotation Types:
    - 'add_type': Add a new type
    - 'update_type': Update an existing type
    - 'delete_type': Delete a type
    - 'add_variable': Add a new variable
    - 'update_variable': Update an existing variable
    - 'delete_variable': Delete a variable
    - 'add_rule': Add a new rule (SMT)
    - 'update_rule': Update an existing rule (SMT)
    - 'delete_rule': Delete a rule
    - 'add_rule_from_nl': Add a rule from natural language
    - 'update_from_rule_feedback': Update rules based on rule feedback
    - 'update_from_scenario_feedback': Update rules based on scenario feedback
    """
    annotation_map = {
        'add_type': {
            'addType': {
                'name': kwargs.get('name'),
                'description': kwargs.get('description'),
                'values': kwargs.get('values', [])
            }
        },
        'update_type': {
            'updateType': {
                'name': kwargs.get('name'),
                'newName': kwargs.get('new_name'),
                'description': kwargs.get('description'),
                'values': kwargs.get('values', [])
            }
        },
        'delete_type': {
            'deleteType': {
                'name': kwargs.get('name')
            }
        },
        'add_variable': {
            'addVariable': {
                'name': kwargs.get('name'),
                'type': kwargs.get('type'),
                'description': kwargs.get('description')
            }
        },
        'update_variable': {
            'updateVariable': {
                'name': kwargs.get('name'),
                'newName': kwargs.get('new_name'),
                'description': kwargs.get('description')
            }
        },
        'delete_variable': {
            'deleteVariable': {
                'name': kwargs.get('name')
            }
        },
        'add_rule': {
            'addRule': {
                'expression': kwargs.get('expression')
            }
        },
        'update_rule': {
            'updateRule': {
                'ruleId': kwargs.get('rule_id'),
                'expression': kwargs.get('expression')
            }
        },
        'delete_rule': {
            'deleteRule': {
                'ruleId': kwargs.get('rule_id')
            }
        },
        'add_rule_from_nl': {
            'addRuleFromNaturalLanguage': {
                'naturalLanguage': kwargs.get('natural_language')
            }
        },
        'update_from_rule_feedback': {
            'updateFromRulesFeedback': {
                'ruleIds': kwargs.get('rule_ids', []),
                'feedback': kwargs.get('feedback')
            }
        },
        'update_from_scenario_feedback': {
            'updateFromScenarioFeedback': {
                'ruleIds': kwargs.get('rule_ids', []),
                'scenarioExpression': kwargs.get('scenario_expression'),
                'feedback': kwargs.get('feedback')
            }
        }
    }
    
    # Validate annotation type
    if annotation_type not in annotation_map:
        raise ValueError(f"Invalid annotation type. Choose from {list(annotation_map.keys())}")
    
    # Validate required parameters
    annotation = annotation_map[annotation_type]
    
    return annotation

In [ ]:
# Check the current Automated Reasoning policy
def get_policy_definition(policy_arn):
    """
    Get the policy definition.
    
    Args:
        policy_arn (str): ARN of the policy.
        
    Returns:
        dict: Policy definition.
    """
    try:
        response = bedrock_client.export_automated_reasoning_policy_version(
            policyArn=policy_arn
        )
        
        return response.get('policyDefinition', {})
    except Exception as e:
        print(f"Error getting policy definition: {str(e)}")
        raise

draft_policy = get_policy_definition(
    policy_arn=policy_arn
)

Let's check what types exist in the Automated Reasoning Policy for the sample medical policy we created in Lab 1. In the first annotation , let's update one of the types programmatically

In [ ]:
draft_policy['types'][1]

# Step 1: Create annotations

Let's create a new type: PatientAgeGroup

In [ ]:

# You can provide a rule using natural language as follows:
nl_rule = 'Provide patients in low-risk with a 24/7 call-back number and clear instructions on warning signs/symptoms that should prompt them to seek immediate medical attention'
nl_rule_annotation = create_annotation('add_rule_from_nl', natural_language=nl_rule)

# Let's update the name of the type "HealthcareFacilityType" to "MedicalFacilityType"
update_type_annotation = create_annotation(
    'update_type', 
    name='HealthcareFacilityType', 
    new_name='MedicalFacilityType', 
    description='The type of healthcare or medical facility where the readmission risk assessment protocol is being implemented', 
    values=[{'value': 'ACUTE_CARE_HOSPITAL',
   'description': 'An acute care hospital with 25 or more beds'},
  {'value': 'CRITICAL_ACCESS_HOSPITAL',
   'description': 'A critical access hospital in a rural area'},
  {'value': 'ACADEMIC_MEDICAL_CENTER',
   'description': 'A university-affiliated medical center that provides education, research, and clinical care'},
  {'value': 'SPECIALTY_HOSPITAL',
   'description': 'A hospital that specializes in specific medical conditions or treatments'},
     {'value': 'EMERGENCY',
   'description': 'Emergency healthcare unit to attend to emergencies before transferring to other hospitals'},
  {'value': 'FACILITY_TYPE_OTHER',
   'description': 'Other types of healthcare facilities not covered by the defined categories'}
   ]
)

# Let's add a new annotation for "PatientAgeGroup"
add_type_annotation = create_annotation(
    'add_type', name='PatientAgeGroup', 
    description='Age Groups of patients for risk classication', 
    values = [
        {'value': 'NEONATES',
         'description': 'New borns who are just born and a maximum of 28 days old'
        },
        {'value': 'INFANTS',
         'description': 'Children who are 29 days to 1 year old'
        },
        {'value': 'TODDLER',
         'description': 'Children who are 1-3 years old'
        }
    ]
    )


# Step 1.1: Scenario Generation

Annotations can be created directly to add/ update or delete rules, variables and types. This is done by analysing the Automated Reasoning policy that has been created by formalizing the logic in the source document when you uploaded the document to Automated Reasoning in Lab 1. 

Scenario generation is a capability that helps you discover which annotations are required through automated testing. In principle, scenario generation replicates the testing process, where you would add a test sample to check if your rules are correct and use that to identify what modifications are required to the Automated Reasoning policy.

In [ ]:
# Let's get the build workflow ID of the ingest content workflow or any successful workflow which contains the updated policy
ar_workflows = bedrock_client.list_automated_reasoning_policy_build_workflows(policyArn=policy_arn)

ar_workflow_ingest_content = [a for a in ar_workflows['automatedReasoningPolicyBuildWorkflowSummaries'] if a['buildWorkflowType'] == 'INGEST_CONTENT']
build_workflow_id = ar_workflow_ingest_content[0]['buildWorkflowId']

In [ ]:
# Generate a scenario until the test sample is meaningful. 
# Run this cell as many times as required! Each time you call 'get_automated_reasoning_policy_next_scenario', it generates a new scenario

scenario = bedrock_client.get_automated_reasoning_policy_next_scenario(policyArn=policy_arn, buildWorkflowId=build_workflow_id)
pprint.pprint(scenario)

In [ ]:
# Add this scenario to test bench
guard_content = scenario['scenario']['alternateExpression']
expected_result = 'SATISFIABLE' # Set this value to VALID, INVALID, SATISFIABLE, IMPOSSIBLE based on the plausibility of the scenario

created_test_case = bedrock_client.create_automated_reasoning_policy_test_case(
    policyArn=policy_arn,
    guardContent=guard_content,
    expectedAggregatedFindingsResult=expected_result,
    clientRequestToken=str(uuid.uuid4()),
    )

In [ ]:
# Run the test case
response = bedrock_client.start_automated_reasoning_policy_test_workflow(
            policyArn=policy_arn,
            buildWorkflowId=build_workflow_id,
            testCaseIds=[created_test_case['testCaseId']],
            clientRequestToken=str(uuid.uuid4()),
        )

In [ ]:
# Check the status to see if the test sample has completed
test_response = bedrock_client.get_automated_reasoning_policy_test_result(
    policyArn=policy_arn,
    buildWorkflowId=build_workflow_id,
    testCaseId=created_test_case['testCaseId'],
    )

if test_response['testResult']['testRunStatus'] == 'COMPLETED':
    if test_response['testResult']['testRunResult'] == 'PASSED':
        print('Test was successful')
    else:
        print(f'Test result is {test_response['testResult']['testRunResult']}')
else:
    print(f'Status of the test is {test_response['testResult']['testRunStatus']}')

If the test was not successful, then we can analyse which rules were involved in testing the scenario and understand what annotations need to be created to update the Automated Reasoning Policy

In [ ]:
def get_ar_policy_rule(policy_arn, rule_id):
    policy_response = policy = bedrock_client.get_automated_reasoning_policy(
            policyArn=policy_arn
        )
    policy_status = policy_response.get('status', 'UNKNOWN')

    # create AR Policy version if not done so already - It is not possible to retrieve policy if its still in DRAFT mode

    # Analyse which rule corresponds to this generated scenario
    ar_policy = bedrock_client.export_automated_reasoning_policy_version(
                policyArn=policy_arn
        )
    
    rule_expression = [r['alternateExpression'] for r in ar_policy['policyDefinition']['rules'] if r['id']==rule_id]
    return rule_expression

In [ ]:
rule_expressions = []
for rule_id in scenario['scenario']['ruleIds']:
    rule_expression = get_ar_policy_rule(policy_arn, rule_id)
    print(f'RuleID: {rule_id}, Expression: {rule_expression}')
    rule_expressions.append(rule_expression)

#### Exercise: if required, create an annotation based on the rules identified from the scenario generation step (Similar to Step 1)

# Step 2: Start the workflow build to refine policy

There is a current service limit of 2 workflows that can be created per Automated Reasoning policy. TO refine the Automated Reasoning policy through annotations, we will need to create a workflow without exceeding this Service Limit. We also need to ensure we retain one successful workflow which can be used to generate scenarios for testing.

In [ ]:
def list_and_delete_AR_build_workflows():
    """
    List and delete workflows based on specific conditions:
    1. Maximum 2 workflows allowed
    2. Delete failed workflows
    3. For completed workflows, check build log of latest and delete if failed
    4. If both completed and build log ok, delete the latest one
    """
    ar_workflows = bedrock_client.list_automated_reasoning_policy_build_workflows(policyArn=policy_arn)
    workflows = ar_workflows['automatedReasoningPolicyBuildWorkflowSummaries']
    
    # Sort workflows by updatedAt timestamp in descending order (latest first)
    sorted_workflows = sorted(workflows, key=lambda x: x['updatedAt'], reverse=True)
    
    for workflow in sorted_workflows:
        # Delete any failed workflows
        if workflow['status'] == 'FAILED':
            bedrock_client.delete_automated_reasoning_policy_build_workflow(
                policyArn=workflow['policyArn'],
                buildWorkflowId=workflow['buildWorkflowId'],
                lastUpdatedAt=workflow['updatedAt']
            )
            continue
            
    # After removing failed workflows, get the list again
    ar_workflows = bedrock_client.list_automated_reasoning_policy_build_workflows(policyArn=policy_arn)
    completed_workflows = [w for w in ar_workflows['automatedReasoningPolicyBuildWorkflowSummaries'] 
                         if w['status'] == 'COMPLETED']
    
    # Sort completed workflows by updatedAt timestamp
    completed_workflows.sort(key=lambda x: x['updatedAt'], reverse=True)
    
    if len(completed_workflows) >= 2:
        # Get latest workflow's build log
        latest_workflow = completed_workflows[0]
        build_log = bedrock_client.get_automated_reasoning_policy_build_workflow_result_assets(
            policyArn=policy_arn,
            buildWorkflowId=latest_workflow['buildWorkflowId'],
            assetType='BUILD_LOG'
        )
        
        # Check if build log indicates failure
        if 'buildLog' in build_log.get('buildWorkflowAssets', {}) and \
           any(entry.get('status') == 'FAILED' 
               for entry in build_log['buildWorkflowAssets']['buildLog'].get('entries', [])):
            # Delete the failed workflow
            bedrock_client.delete_automated_reasoning_policy_build_workflow(
                policyArn=latest_workflow['policyArn'],
                buildWorkflowId=latest_workflow['buildWorkflowId'],
                lastUpdatedAt=latest_workflow['updatedAt']
            )
        else:
            # If build log is ok but we still have 2 completed workflows,
            # delete the latest one to maintain the limit
            bedrock_client.delete_automated_reasoning_policy_build_workflow(
                policyArn=latest_workflow['policyArn'],
                buildWorkflowId=latest_workflow['buildWorkflowId'],
                lastUpdatedAt=latest_workflow['updatedAt']
            )

In [ ]:
build_workflow_type = "REFINE_POLICY"

current_policy = bedrock_client.export_automated_reasoning_policy_version(
    policyArn=policy_arn
)

# We pass in the current policy definition, and the annotation as part of the PolicyRepairAsset
# The annotation is applied to the current policy, to build a new policy
def apply_annotation_through_policy_repair_asset():
    return bedrock_client.start_automated_reasoning_policy_build_workflow(
                policyArn=policy_arn,
                buildWorkflowType=build_workflow_type,
                clientRequestToken=str(uuid.uuid4()),
                sourceContent={
                    'policyDefinition': {
                        'rules': current_policy['policyDefinition']['rules'],
                        'variables': current_policy['policyDefinition']['variables'],
                        'types': current_policy['policyDefinition']['types'],
                        'version': current_policy['policyDefinition']['version']
                    },
                    'workflowContent': {
                        'policyRepairAssets': {
                            'annotations': [add_type_annotation]
                        }
                    }
                }
            )
    

try:
    reponse = apply_annotation_through_policy_repair_asset()
except Exception as e:
    print(f'Error: {e}')
    print('Clearing failed workflows from queue')

    list_and_delete_AR_build_workflows()

    print('Trying to create annotation through a new workflow')
    reponse = apply_annotation_through_policy_repair_asset()

In [ ]:
# Check the queue of workflows to see if the REFINE_POLICY workflow has been created and is being built
ar_workflows = bedrock_client.list_automated_reasoning_policy_build_workflows(policyArn=policy_arn)
ar_workflows

### Let's retrieve the Policy Definition from the latest workflow and check if the annotation of PatientAgeGroup exists

In [ ]:
def get_latest_build_workflow(policy_arn):
    """
    Retrieve the latest build workflow for a given policy
    
    Args:
        policy_arn (str): ARN of the Automated Reasoning Policy
    
    Returns:
        dict: Latest build workflow details
    """
    try:
        workflows_response = bedrock_client.list_automated_reasoning_policy_build_workflows(
            policyArn=policy_arn
        )
        
        # Sort workflows by updatedAt timestamp in descending order
        sorted_workflows = sorted(
            workflows_response['automatedReasoningPolicyBuildWorkflowSummaries'], 
            key=lambda x: x.get('updatedAt', ''), 
            reverse=True
        )
        
        # Return the most recent workflow
        return sorted_workflows[0] if sorted_workflows else None
    
    except Exception as e:
        print(f"Error retrieving build workflows: {str(e)}")
        raise

def get_workflow_result_assets(policy_arn, build_workflow_id, asset_type='POLICY_DEFINITION'):
    """
    Retrieve result assets for a specific build workflow
    
    Args:
        policy_arn (str): ARN of the Automated Reasoning Policy
        build_workflow_id (str): ID of the build workflow
        asset_type (str, optional): Type of asset to retrieve. 
            Defaults to 'POLICY_DEFINITION'
            Options: 'POLICY_DEFINITION', 'QUALITY_REPORT', 'BUILD_LOG'
    
    Returns:
        dict: Workflow result assets
    """
    try:
        result_assets_response = bedrock_client.get_automated_reasoning_policy_build_workflow_result_assets(
            policyArn=policy_arn,
            buildWorkflowId=build_workflow_id,
            assetType=asset_type
        )
        
        return result_assets_response.get('buildWorkflowAssets', {})
    
    except Exception as e:
        print(f"Error retrieving workflow result assets: {str(e)}")
        raise

def get_latest_workflow_results(policy_arn, build_workflow_id=None,asset_type='POLICY_DEFINITION'):
    """
    Comprehensive method to get the latest workflow results
    
    Args:
        policy_arn (str): ARN of the Automated Reasoning Policy
        build_workflow_id (str, optional): None if not passed in. It gets the latest workflow ID if not passed
        asset_type (str, optional): Type of asset to retrieve
    
    Returns:
        dict: Latest workflow result assets
    """
    # Get the latest build workflow
    if build_workflow_id is None:
        latest_workflow = get_latest_build_workflow(policy_arn)
    
        if not latest_workflow:
            print("No build workflows found.")
            return None
        build_workflow_id = latest_workflow['buildWorkflowId']
    # Get workflow result assets
    workflow_results = get_workflow_result_assets(
        policy_arn, 
        build_workflow_id, 
        asset_type
    )
    
    return workflow_results

In [ ]:
policy_definitions = get_latest_workflow_results(
    policy_arn=policy_arn, 
    # build_workflow_id='90aaf8fb-015a-42fb-9720-e87f36854989', 
    asset_type='POLICY_DEFINITION'
    )

In [ ]:
for type in policy_definitions['policyDefinition']['types']:
    print(type)

### We can see that the type (PatientAgeGroup) we included has been added to the newly built Automated Reasoning Policy!

## Quality Report

This is a useful report which helps you identify which types, values and variables are unused in the Automated Reasoning Policy. It also shows any conflicting rules or disjoint rule setsto help you identify which rules, variables and types need SME attention.

In [ ]:
quality_result = get_latest_workflow_results(
    policy_arn=policy_arn, 
    # build_workflow_id='90aaf8fb-015a-42fb-9720-e87f36854989', 
    asset_type='QUALITY_REPORT'
    )

In [ ]:
quality_result

### Build Log
This provides helpful details to identify the status of the annotation that was applied through the REFINE_POLICY workflow

In [ ]:
build_log = get_latest_workflow_results(
    policy_arn=policy_arn, 
    # build_workflow_id='90aaf8fb-015a-42fb-9720-e87f36854989', 
    asset_type='BUILD_LOG'
    )

In [ ]:
# The build log shows the status of the individual annotations and the steps taken to build that annotation
build_log

# Step 3: Update the Automated Reasoning Policy with the New Policy generated in Step 2

In [ ]:
# Let's now update the policy with the workflow assets that have been created
# Update the policy definition with the new annotation
response = bedrock_client.update_automated_reasoning_policy(
    policyArn=policy_arn,
    policyDefinition={
        'rules': policy_definitions['policyDefinition']['rules'],
        'variables': policy_definitions['policyDefinition']['variables'],
        'types': policy_definitions['policyDefinition']['types'],
    }
)

In [ ]:
response

In [ ]:
# Retrieve latest policy and check that the update exists
current_policy = bedrock_client.export_automated_reasoning_policy_version(
    policyArn=policy_arn
)

for type in current_policy['policyDefinition']['types']:
    print(type)


# Success!
We can see that the latest Automated Reasoning Policy contains the type we added - PatientAgeGroup!